In [139]:
# import pandas lib as pd
import pandas as pd
# read by default 1st sheet of an excel file
df= pd.read_excel('datatest.xlsx',header=0, sheet_name="Style1")
row_name=df.columns.values
df.insert(2,'sewing',0,True)
df.insert(3,'wating',df['headcount'],True)


In [140]:

def update_sequence(df_style,point_check):
    sequence=point_check[1]
    sew_collect=point_check[2]
    df_style.loc[sequence-1,'sewing']-=sew_collect
    df_style.loc[sequence-1,'wating']+=sew_collect
    df_style.loc[sequence-1,'WIP']+=sew_collect
    
def cal_time_check(df_style,time,num_seq,sew,list_point_check):
    output=[]
    point_time=time + df_style.loc[num_seq,'SAM']
    output.append(round(point_time,1))
    output.append(num_seq+1)
    output.append(sew)
    list_point_check.append(output)
    list_point_check.sort()
    
def cal_wip(df_style,num_seq,time,list_point_check):
    # num_seq=i ~ row=i-1
    if df_style.loc[num_seq,'wating'] > 0 and df_style.loc[num_seq-1,'WIP'] > 0 and df_style.loc[num_seq-1,'WIP'] <= df_style.loc[num_seq,'wating']:
        cal_time_check(df_style,time,num_seq,df_style.loc[num_seq-1,'WIP'],list_point_check)
        df_style.loc[num_seq,'wating']-=df_style.loc[num_seq-1,'WIP']
        df_style.loc[num_seq,'sewing']+=df_style.loc[num_seq-1,'WIP']
        df_style.loc[num_seq-1,'WIP']=0
        
    if df_style.loc[num_seq,'wating']> 0 and df_style.loc[num_seq-1,'WIP']> df_style.loc[num_seq,'wating']:
        cal_time_check(df_style,time,num_seq,df_style.loc[num_seq,'wating'],list_point_check)
        df_style.loc[num_seq-1,'WIP']-=df_style.loc[num_seq,'wating']
        df_style.loc[num_seq,'wating']=0
        df_style.loc[num_seq,'sewing']=df_style.loc[num_seq,'headcount']
     
def sequence_first(df_style,ti_step,ti_line,num_seq,list_point_check):
    df_style.loc[num_seq-1,'wating']=0
    df_style.loc[num_seq-1,'sewing']=4
    cycle=ti_line//df_style.loc[num_seq-1,'SAM']
    if ti_step==0:
        for cyc in range(0,int(cycle)):
            cal_time_check(df_style,cyc*df_style.loc[num_seq-1,'SAM'],num_seq-1,df_style.loc[num_seq-1,'headcount'],list_point_check)
            # print(time)
            # print(".")
        
    # print(time)
    # print(df['WIP'][0])

if __name__=="__main__":
    list_check=[]
    time_step=0
    time_line=100
    for i in range(1,df.shape[0]):
        cal_wip(df,i,time_step,list_check)
    sequence_first(df,time_step,time_line,1,list_check)
    # print(time_step)
    # print(df)
    # for _ in list_check:
    #     print(_)
    pcheck=list_check.pop(0)
    time_step=pcheck[0]
    # print(pcheck)
    # print(time_step)
    while time_step<= time_line:
        update_sequence(df,pcheck)
        sequence_first(df,time_step,time_line,1,list_check)
        
        for i in range(1,df.shape[0]):
            cal_wip(df,i,time_step,list_check)
        # print(time_step)
        # print(df)
        # for _ in list_check:
        #     print(_)
        try:
            pcheck=list_check.pop(0)
            time_step=pcheck[0]
        except:
            break
        # print(pcheck)
        # print(time_step)
print(time_line)
print(df)

100
   sequence    operation  sewing  wating   SAM  headcount  WIP
0         1   Bind Panel       4       0   3.3          4   28
1         2    Over Edge       4       0   3.6          4    0
2         3   Seam Panel      16       4  15.0         20   37
3         4  Seam Crotch       7       0   7.4          7    5
4         5     Hem Legs       8       0   8.6          8    0
5         6     Sew Band       8       1   9.2          9   86


In [117]:
# import pandas lib as pd
import pandas as pd
# read by default 1st sheet of an excel file
df2= pd.read_excel('datatest.xlsx',header=0, sheet_name="Style2")
row_name=df2.columns.values
df2.insert(2,'sewing',0,True)
df2.insert(3,'wating',df2['headcount'],True)
df2

,sequence,operation,sewing,wating,SAM,headcount,WIP
0,1,Bind Panel,0,4,3.3,4,7
1,2,Over Edge,0,4,3.6,4,25
2,3,Attach Sling Rubber,0,7,6.0,7,10
3,4,Sew Center,0,8,8.0,8,14
4,5,Topstitch,0,6,4.0,6,8
5,6,Seam Panel,0,20,15.0,20,18
6,7,Seam Crotch,0,7,7.4,7,10
7,8,Hem Legs,0,8,8.6,8,6
8,9,Sew Band,0,9,9.2,9,0


In [118]:
# import pandas lib as pd
import pandas as pd
# read by default 1st sheet of an excel file
df3= pd.read_excel('datatest.xlsx',header=0, sheet_name="OPERATION_CONFIG")
row_name=df3.columns.values
df3

,style,operation,next operation
0,Style 1,Bind Panel,Seam Panel
1,Style 2,Bind Panel,Sew Center
2,Style 2,Over Edge,Sew Center
3,Style 2,Sew Center,Topstitch
4,Style 2,Attach Sling Rubber,Topstitch


In [119]:
# import pandas lib as pd
import pandas as pd
# read by default 1st sheet of an excel file
df4= pd.read_excel('datatest.xlsx',header=0, sheet_name="NO_WIP_REQ")
row_name=df4.columns.values
df4

,style,operation
0,Style 1,Bind Panel
1,Style 1,Over Edge
2,Style 2,Bind Panel
3,Style 2,Over Edge
4,Style 2,Attach Sling Rubber


In [141]:
def check_sequence_first(df_no_wip_req, name_style):
    list_seq_first=df_no_wip_req[df_no_wip_req["style"]==name_style]["operation"].values
    return list_seq_first

def check_sequence_depend(df_config, name_style):
    list_seq_depend=df_config[df_config["style"]==name_style]["next operation"].unique()
    return list_seq_depend

def check_sequence(df_style,list_seq_first,list_seq_depend,ti_step,ti_line,num_seq,list_point_check):
    name_sequence=df_style.loc[num_seq-1,"operation"]
    if name_sequence in list_seq_first:
        sequence_first(ti_step,ti_line,num_seq,list_point_check)
    if name_sequence in list_seq_depend:
        pass
    

    

In [142]:
list_first= check_sequence_first(df4,"Style 2")
print(list_first)
list_depend=check_sequence_depend(df3,"Style 2")
print(list_depend)

['Bind Panel' 'Over Edge' 'Attach Sling Rubber']
['Sew Center' 'Topstitch']


In [143]:
check_sequence(df2,1,list_first,list_depend)

TypeError: check_sequence() missing 3 required positional arguments: 'ti_line', 'num_seq', and 'list_point_check'